In [1]:
from datasets import load_dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

c:\Users\devan\anaconda3\envs\test\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\devan\.cache\huggingface\hub\datasets--cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [2]:
from transformers import BartTokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

c:\Users\devan\anaconda3\envs\test\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\devan\.cache\huggingface\hub\models--facebook--bart-large-cnn. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

c:\Users\devan\anaconda3\envs\test\Lib\site-packages\transformers\tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [7]:
import nltk

# Download ALL required NLTK data
nltk.download("punkt", download_dir="./nltk_data")
nltk.download("punkt_tab", download_dir="./nltk_data")  # Explicitly download punkt_tab
nltk.download("wordnet", download_dir="./nltk_data")    # Additional dependency

# Configure NLTK to use custom path PERMANENTLY
nltk.data.path = ["./nltk_data"] + nltk.data.path

[nltk_data] Downloading package punkt to ./nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package punkt_tab to ./nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package wordnet to ./nltk_data...


In [8]:
def create_extractive_labels(example):
    # Force NLTK to use custom path
    nltk.data.path = ["./nltk_data"] + nltk.data.path
    
    article_sentences = nltk.sent_tokenize(example["article"])
    summary = example["highlights"]
    labels = []
    
    for sent in article_sentences:
        score = scorer.score(summary, sent)['rougeL'].fmeasure
        labels.append(1 if score > 0.5 else 0)
    
    example["labels"] = labels
    example["sentences"] = article_sentences
    return example

In [9]:
import nltk
from rouge_score import rouge_scorer

# Download punkt to a custom directory
nltk.download('punkt', download_dir='./nltk_data')
nltk.data.path.append('./nltk_data')  # Add to NLTK's search path

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

# Define the function with explicit NLTK path
def create_extractive_labels(example):
    nltk.data.path = ['./nltk_data'] + nltk.data.path  # Force custom path
    article_sentences = nltk.sent_tokenize(example["article"])
    summary = example["highlights"]
    labels = []
    
    for sent in article_sentences:
        score = scorer.score(summary, sent)['rougeL'].fmeasure
        labels.append(1 if score > 0.5 else 0)
    
    example["labels"] = labels
    example["sentences"] = article_sentences
    return example

# Apply to dataset
labeled_dataset = dataset.map(create_extractive_labels)

[nltk_data] Downloading package punkt to ./nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
from transformers import BartForConditionalGeneration, TrainingArguments, Trainer

model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=3e-5,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
)
trainer.train()

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer, TrainingArguments, Trainer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_extractive(examples):
    return tokenizer(examples["sentences"], padding="max_length", truncation=True)

extractive_dataset = labeled_dataset.map(tokenize_extractive, batched=True)

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results_extractive",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    learning_rate=2e-5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=extractive_dataset["train"],
    eval_dataset=extractive_dataset["validation"],
)
trainer.train()

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

def evaluate_summary(predicted, reference):
    return scorer.score(predicted, reference)

# Apply to test dataset
abstractive_scores = []
extractive_scores = []

for example in dataset["test"]:
    # Generate abstractive summary
    abstractive_summary = generate_abstractive(example["article"])
    # Generate extractive summary
    extractive_summary = generate_extractive(example["article"])
    # Compute scores
    abstractive_scores.append(evaluate_summary(abstractive_summary, example["highlights"]))
    extractive_scores.append(evaluate_summary(extractive_summary, example["highlights"]))

# Average scores
avg_abstractive_rouge = {key: sum(s[key] for s in abstractive_scores)/len(abstractive_scores) for key in ['rouge1', 'rougeL']}
avg_extractive_rouge = {key: sum(s[key] for s in extractive_scores)/len(extractive_scores) for key in ['rouge1', 'rougeL']}